# Model Order Reduction & Machine Learning - Project
## Nonlinear Elliptic problem (NEP) **PINN**

### Importing library

In [ ]:
!pip install pandas

In [ ]:
import sys
sys.path.append('../../CppToPython')

import numpy as np
import GeDiM4Py as gedim
from scipy.sparse.linalg import splu
import time
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import torch
import torch.nn as nn
from torch.autograd import Variable

# Run to avoid to many warnings
import warnings
warnings.filterwarnings('ignore')

import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR
import os
import pandas as pd

### Initialize

In [ ]:
lib = gedim.ImportLibrary("../../CppToPython/release/GeDiM4Py.so")

config = { 'GeometricTolerance': 1.0e-8 }
gedim.Initialize(config, lib)

np.random.seed(23)

## PART 2
Let us consider the two-dimensional spatial domain $\Omega = (0, 1)^2$. We want to solve the following parametrized problem: given $\mu = (\mu_0, \mu_1) \in \mathcal{P} = [0.1, 1]^2$, find $u(\mu)$ such that
$$
-\Delta u(\mu) + \frac{\mu_0}{\mu_1} (e^{\mu_1 u(\mu)} - 1) = g(x;\mu),
$$
with homogeneous Dirichlet condition on the boundary, i.e. zero-boundary condition.

The forcing term, that does not depend on the parameters, defined as
$$
g(x;\mu) = g_1 = 100 \sin(2\pi x_0) \cos(2\pi x_1) \quad \forall x = (x_0, x_1) \in \Omega.
$$

**Tasks:**

1. Solve the problem by means of POD-Galerkin method over a Finite Element full order model -> using high fidelity element approximation of POD
2. Solve the problem with a parametric PINN -> find the NN structure to reduce the problem
3. Compare the two approaches in terms of computational costs and accuracy with respect to the full order model -> compare in terms of execution time & error
4. **Optional:** Solve the problem with the POD-NN approach and compare it to the other two strategies.

In [ ]:
# Points for the diffusion term
def Elliptic_a(numPoints, points): 
	values_a = np.ones(numPoints, order='F') # Fortran order of memorization of the matrix: column by column
	return values_a.ctypes.data

# Points for the reaction term
def Elliptic_c(numPoints, points):
	values_c = np.ones(numPoints, order='F') 
	return values_c.ctypes.data    

# Non linear reaction term
# numPoints := quadrature points
def Elliptic_non_linear_c(numPoints, points, u, u_x, u_y):
	vec_u = gedim.make_nd_array(u, numPoints, np.double) # Evaluation of the function in the quadrature points numPoints
	values_nl_c = vec_u
	return values_nl_c.ctypes.data

def Elliptic_non_linear_der_f(numPoints, points, u, u_x, u_y):
    # Converts u_x and u_y pointers or arrays to NumPy arrays
	vecu_x = gedim.make_nd_array(u_x, numPoints, np.double)
	vecu_y = gedim.make_nd_array(u_y, numPoints, np.double)
    
	values_nl_d_f = np.zeros((2, numPoints), order='F')

    # Inserts the partial derivatives of u into the new array values_nl_d_f
	values_nl_d_f[0,:] = vecu_x
	values_nl_d_f[1,:] = vecu_y
    
	return values_nl_d_f.ctypes.data

# Nonlinear reaction term
def Elliptic_non_linear_f(numPoints, points, u, u_x, u_y): 
    vec_u = gedim.make_nd_array(u, numPoints, np.double)
    vecu_x = gedim.make_nd_array(u_x, numPoints, np.double)
    vecu_y = gedim.make_nd_array(u_y, numPoints, np.double)
    
    values_nl_f = (mu_0/mu_1) * (np.exp(mu_1*vec_u)-1) # Computation of f
    
    return values_nl_f.ctypes.data

def Elliptic_exactSolution(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)

    # Definition of u
	values_ex = mu_0 * (matPoints[1,:] * (1.0 - matPoints[1,:]) * matPoints[0,:] * (1.0 - matPoints[0,:]))
    
	return values_ex.ctypes.data

def Elliptic_exactDerivativeSolution(direction, numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)

	if direction == 0: # wrt x
		values_ex_d = mu_0 * (1.0 - 2.0 * matPoints[0,:]) * matPoints[1,:] * (1.0 - matPoints[1,:]) # 16⋅(1−2x)⋅y⋅(1−y)
	elif direction == 1: # wrt y
		values_ex_d = mu_0 * (1.0 - 2.0 * matPoints[1,:]) * matPoints[0,:] * (1.0 - matPoints[0,:]) # 16⋅(1−2y)⋅x⋅(1−x)
	else:
		values_ex_d = np.zeros(numPoints, order='F')

	return values_ex_d.ctypes.data

# Forcing term for evaluating the error
def Elliptic_exactSolution_g(numPoints, points):
    matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
    
    # Definition of u
    # values_ex = 2*mu_0*(y*(1-y)+x*(1-x)) + (mu_0/mu_1)*(np.exp(mu_0*mu_1*x*y*(1-x)(1-y)) -1)
    values_ex = 2*mu_0*(matPoints[1,:]*(1-matPoints[1,:])+matPoints[0,:]*(1-matPoints[0,:])) + (mu_0/mu_1)*(np.exp(mu_0*mu_1*matPoints[0,:]*matPoints[1,:]*(1-matPoints[0,:])*(1-matPoints[1,:])) -1)
    
    return values_ex.ctypes.data

def Ones(numPoints, points):
	values_one = np.ones(numPoints, order='F')
	return values_one.ctypes.data

def OnesDerivative(numPoints, points):
	values_one_d = np.ones((2, numPoints), order='F')
	return values_one_d.ctypes.data

def Zeros(numPoints, points):
	values_zero = np.zeros(numPoints, order='F')
	return values_zero.ctypes.data

def ZerosDerivative(direction, numPoints, points):
	values_zero_d = np.zeros(numPoints, order='F')
	return values_zero_d.ctypes.data

# Definition of the forcing term g_1
def Elliptic_g_1(numPoints, points): 
    matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double) # X
    # g_1(X,mu) = 100*sin(2*mu_0*pi*x)*cos(2*mu_0*pi*y), X=(x,y)
    values_g = 100*np.sin(2*mu_0*np.pi*matPoints[0,:])*np.cos(2*mu_0*np.pi*matPoints[1,:])
    return values_g.ctypes.data

#### Mesh and domain definition

In [ ]:
order = 1 # Start from 1, at the END we will decide if we have to improve the order
P = [0.1, 1.] # Parametric space

In [ ]:
# Fix the right domain
mesh_size = 0.00312

# Updating the domain wrt the mesh_size
domain = { 'SquareEdge': 1.0, 'VerticesBoundaryCondition': [1,1,1,1], 'EdgesBoundaryCondition': [1,1,1,1], 'DiscretizationType': 1, 'MeshCellsMaximumArea': mesh_size } 
[meshInfo, mesh] = gedim.CreateDomainSquare(domain, lib) 
discreteSpace = { 'Order': order, 'Type': 1, 'BoundaryConditionsType': [1, 2] } # 2 := Dirichlet boundary condition
[problemData, dofs, strongs] = gedim.Discretize(discreteSpace, lib)

In [ ]:
# Initialization of the guess of the solution --> the NM converges rapidly if we start not far away from the solution
u_k = np.zeros(problemData['NumberDOFs'], order='F') 
u_strong = np.zeros(problemData['NumberStrongs'], order='F') # Vector containing the strongly imposed components of the solution, ie Dirichlet conditions

#### Newton method for high fidelity snapshots computation - TO COMPUTE THE ERROR

In [ ]:
sample_mu_0 = np.random.uniform(0.1, 1, 300) # Uniform sampling in mu_Rang of 100 samples
sample_mu_1 = np.random.uniform(0.1, 1, 300)

snapshots_matrix = []
convergence_history = []  
max_iterations = 100
newton_tol = 1e-8
sol_quad_hf = []

for mu_0, mu_1 in zip(sample_mu_0, sample_mu_1):
    
    # Set Newton parameter at each change of the parameter
    du_norm = 1.0
    u_k_norm = 1.0
    num_iteration = 1
    du_norms_for_sample = []
    num_iteration = 1
    rel_error = 0

    # Iterative step - Setting a relative tollerance
    while num_iteration < max_iterations and du_norm > newton_tol * u_k_norm: 

        # 1- Left side of the equation
        
        # Linear
        [stiffness, stiffnessStrong] = gedim.AssembleStiffnessMatrix(Elliptic_a, problemData, lib) # Indipendent from mu_0 & mu_1
        # Non linear 
        [reaction, reactionStrong] = gedim.AssembleNonLinearReactionMatrix(Elliptic_c, Elliptic_non_linear_c, u_k, u_strong, problemData, lib)
       
        # 2- Right hand side of the function
        
        # Linear part
        forcingTerm_g = gedim.AssembleForcingTerm(Elliptic_g_1, problemData, lib)
        # Non linear part
        forcingTerm_v = gedim.AssembleNonLinearForcingTerm(Ones, Elliptic_non_linear_f, u_k, u_strong, problemData, lib)
        forcingTerm_der_v = gedim.AssembleNonLinearDerivativeForcingTerm(OnesDerivative, Elliptic_non_linear_der_f, u_k, u_strong, problemData, lib)
    
        # Solving with the LU solver because we're in a generic setting (no idea of the structure of the matrix)
        # CAN WE DONE BETTER?
        du = gedim.LUSolver(stiffness + reaction, \
                forcingTerm_g - forcingTerm_v - forcingTerm_der_v, \
                lib)
        
        u_k = u_k + du # Update
        
        # Compute norm for stopping criterium
        du_norm = gedim.ComputeErrorL2(Zeros, du, np.zeros(problemData['NumberStrongs'], order='F'), lib)  # ||du|| = ||u(k) - u(k-1)|| L^2-norm
        du_norms_for_sample.append(du_norm)
        u_k_norm = gedim.ComputeErrorL2(Zeros, u_k, u_strong, lib) # ||u_k|| L^2-norm

    
        # Compute the relative error of two consecutive iterations
        rel_error = du_norm / u_k_norm       
        
        num_iteration = num_iteration + 1

    snapshots_matrix.append(np.copy(u_k))
    convergence_history.append(du_norms_for_sample)

    # Compute the sol on the quadrature points
    [numQuadraturePoints, quadraturePoints, quadratureWeights, solution_hf, solution_x_hf, solution_y_hf] = gedim.EvaluateSolutionOnPoints(u_k, u_strong, lib)
    # append solution on qudrature points
    sol_quad_hf.append(solution_hf)

    # Print at the end of the iteration before the change of the parameter
    #print(f"{'mu_0':<10} {'mu_1':<10} {'iters':<6} {'rel_err_L2':<14}")
    #print(f"{mu_0:<10.4f} {mu_1:<10.4f} {(num_iteration-1):<6d} {rel_error:<14.2e}")

snapshots_matrix = np.array(snapshots_matrix)
print(f"snapshots_matrix.shape: {snapshots_matrix.shape}")

# Just to check the final u_k, mu_0, mu_1
last_u_k_exact = u_k

In [ ]:
# Plot and show the solution
gedim.PlotSolution(mesh, dofs, strongs, last_u_k_exact, u_strong)
[numQuadraturePoints, quadraturePoints, quadratureWeights, sol, sol_x, sol_y] = gedim.EvaluateSolutionOnPoints(last_u_k_exact, u_strong, lib)
gedim.ExportSolutionOnPoints(numQuadraturePoints, quadraturePoints, last_u_k_exact, lib)
print(f"quadratureWeights: {quadratureWeights}")

gedim.ExportSolutionOnPoints(numQuadraturePoints, quadraturePoints, sol, lib) # To export solutions on paraview

## 2. PINN

In [ ]:
# Just for the first epoch
def generate_data(N_domain=1750, N_boundary=150):
    # Points in the domain
    x_in = torch.rand((N_domain, 2))  # [x0, x1] ∈ Ω
    mu_in = (torch.rand((1, 2)) * 0.9 + 0.1).expand(N_domain, -1) # [mu0, mu1] ∈ [0.1, 1]^2

    # Points on the border of the domain (edge of the square)
    x0 = torch.rand(N_boundary, 1)
    zeros = torch.zeros_like(x0)
    ones = torch.ones_like(x0)

    xb = torch.cat([
        torch.cat([x0, zeros], dim=1),
        torch.cat([x0, ones], dim=1),
        torch.cat([zeros, x0], dim=1),
        torch.cat([ones, x0], dim=1)
    ], dim=0)

    mu_b = mu_in[:N_boundary*4, :]
    return x_in, mu_in, xb, mu_b

In [ ]:
# For all the other epochs, it samples on the point on the domain in which the loss is higher so that he can learn better
def adaptive_sample(model, N_tot=5000, N_sel=1750, N_boundary=150):
    X = torch.rand(N_tot, 2, device=device)
    M = (torch.rand(N_tot, 2, device=device) * 0.9 + 0.1)

    # Compute the residual
    X.requires_grad_(True)
    R_vals = R(X, M, model).detach().abs().cpu().numpy().flatten()  # .detach() spezza il grafo

    idx = np.argsort(R_vals)[-N_sel:]
    x_int = X.detach()[idx]  
    mu_int = M.detach()[idx]

    # Point on the border
    x0 = torch.rand(N_boundary, 1, device=device)
    zeros = torch.zeros_like(x0)
    ones = torch.ones_like(x0)
    xb = torch.cat([
        torch.cat([x0, zeros], 1),
        torch.cat([x0, ones], 1),
        torch.cat([zeros, x0], 1),
        torch.cat([ones, x0], 1)
    ], 0)
    mu_b = (torch.rand(1, 2, device=device) * 0.9 + 0.1).expand(xb.shape[0], -1)

    return x_int, mu_int, xb, mu_b

In [ ]:
# x: [N, 2], mu: [N, 2]
def R(x, mu, net):
    x.requires_grad_(True)
    mu.requires_grad_(False)

    u = net(x, mu)  # output shape: [N, 1]

    # Compute the gradient ∇u (i.e. du/dx e du/dy)
    grads = torch.autograd.grad(u.sum(), x, create_graph=True)[0]  # shape [N, 2]
    u_x = grads[:, 0].unsqueeze(1)  # du/dx
    u_y = grads[:, 1].unsqueeze(1)  # du/dy

    # Laplacian = d²u/dx² + d²u/dy²
    u_xx = torch.autograd.grad(u_x.sum(), x, create_graph=True)[0][:, 0].unsqueeze(1)
    u_yy = torch.autograd.grad(u_y.sum(), x, create_graph=True)[0][:, 1].unsqueeze(1)
    laplacian_u = u_xx + u_yy

    # Nonlinear term
    mu0 = mu[:, 0].unsqueeze(1)
    mu1 = mu[:, 1].unsqueeze(1)
    nonlinear = (mu0 / mu1) * (torch.exp(mu1 * u) - 1)

    # Forcing term g(x)
    x0 = x[:, 0].unsqueeze(1)
    x1 = x[:, 1].unsqueeze(1)
    g = 100 * torch.sin(2 * torch.pi * x0) * torch.cos(2 * torch.pi * x1)

    # Residuo della PDE
    residual = -laplacian_u + nonlinear - g

    return residual

#### **Train** computed on Kaggle (see the result in a different folder).

In [ ]:
def train_plot(model, epochs=50000, lr=1e-3, lambda_bc=1.0, patience=7500, delta=1e-4, batch_size=None, save_path="./models"):
    os.makedirs(save_path, exist_ok=True)

    model.to(device)
    mse_loss = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_phys_loss = float('inf')
    best_model_state = None
    best_model_path = None
    epochs_no_improve = 0
    using_lbfgs = False

    loss_history_total = []
    loss_history_phys = []
    loss_history_bc = []
    times_per_epoch = []

    for epoch in range(epochs):
        start_time = time.time()
        model.train()
        x_in, mu_in, xb, mu_b = generate_data()

        if epoch != 0:
            x_in, mu_in, xb, mu_b = adaptive_sample(model)

        x_in = x_in.to(device)
        mu_in = mu_in.to(device)
        xb = xb.to(device)
        mu_b = mu_b.to(device)

        try:
            if not using_lbfgs:
                optimizer.zero_grad()
                loss_phys = mse_loss(R(x_in, mu_in, model), torch.zeros_like(x_in[:, :1]))
                u_b = model(xb, mu_b)
                loss_bc = mse_loss(u_b, torch.zeros_like(u_b))

                loss_total = loss_phys + lambda_bc * loss_bc

                if torch.isnan(loss_total):
                    raise ValueError("NaN detected in Adam loss computation")

                loss_total.backward()
                optimizer.step()

                if loss_phys.item() + delta < best_phys_loss:
                    best_phys_loss = loss_phys.item()
                    best_model_state = model.state_dict()
                    best_model_path = os.path.join(save_path, f"model_best.pth")
                    torch.save(best_model_state, best_model_path)
                    torch.save(best_model_state, "PINNResNet.pth")
                    epochs_no_improve = 0
                else:
                    epochs_no_improve += 1

                # Interrompe l'allenamento se la total loss è < 1e-4
                if loss_total.item() < 1e-4:
                    final_model_path = os.path.join(save_path, "model_total_loss_e-4_PINNResNet.pth")
                    torch.save(model.state_dict(), final_model_path)
                    print(f"\n💾 Total loss reached e-4 at epoch {epoch}. Model saved at {final_model_path}")
                    break

                if loss_bc.item() < 1e-1 and loss_phys.item() < 1e-1:
                    print(f"\nSwitching to LBFGS at epoch {epoch}")
                    using_lbfgs = True
                    optimizer = torch.optim.LBFGS(model.parameters(), lr=1.0, max_iter=500)
                    if best_model_state:
                        model.load_state_dict(best_model_state)

            else:
                def closure():
                    optimizer.zero_grad()
                    loss_phys_cl = mse_loss(R(x_in, mu_in, model), torch.zeros_like(x_in[:, :1]))
                    u_b_cl = model(xb, mu_b)
                    loss_bc_cl = mse_loss(u_b_cl, torch.zeros_like(u_b_cl))
                    loss_total_cl = loss_phys_cl + lambda_bc * loss_bc_cl

                    if torch.isnan(loss_total_cl):
                        raise ValueError("NaN detected in LBFGS loss computation")

                    loss_total_cl.backward()
                    return loss_total_cl

                optimizer.step(closure)

                loss_phys = mse_loss(R(x_in, mu_in, model), torch.zeros_like(x_in[:, :1]))
                u_b = model(xb, mu_b)
                loss_bc = mse_loss(u_b, torch.zeros_like(u_b))
                loss_total = loss_phys + lambda_bc * loss_bc

                # Interrompe l'allenamento se la total loss è < 1e-4
                if loss_total.item() < 1e-3:
                    final_model_path = os.path.join(save_path, "model_total_loss_e-4_PINNResNet.pth")
                    torch.save(model.state_dict(), final_model_path)
                    print(f"\n💾 Total loss reached e-4 at epoch {epoch}. Model saved at {final_model_path}")
                    break

        except ValueError as e:
            print(f"\n❌ Training stopped due to error at epoch {epoch}: {str(e)}")
            if best_model_state:
                print("🔄 Reverting to best model state before NaN.")
                model.load_state_dict(best_model_state)
                torch.save(best_model_state, "PINNResNet.pth")
                nan_model_path = os.path.join(save_path, f"model_nan_epoch{epoch}.pth")
                torch.save(model.state_dict(), nan_model_path)
            break

        loss_history_total.append(loss_total.item())
        loss_history_phys.append(loss_phys.item())
        loss_history_bc.append(loss_bc.item())
        times_per_epoch.append(time.time() - start_time)

        if epoch % 500 == 0 or epoch == epochs - 1:
            opt_name = "Adam" if not using_lbfgs else "LBFGS"
            print(f"[{opt_name}] Epoch {epoch} - Total: {loss_total.item():.2e} | PDE: {loss_phys.item():.2e} | BC: {loss_bc.item():.2e}")

        if not using_lbfgs and epochs_no_improve >= patience:
            print(f"\n🛑 Early stopping at epoch {epoch} | Best PDE loss: {best_phys_loss:.2e}")
            model.load_state_dict(best_model_state)
            break

    # Salva comunque il modello finale
    final_model_path = os.path.join(save_path, "model_final.pth")
    torch.save(model.state_dict(), final_model_path)

    # Plotta le perdite
    plt.figure(figsize=(10, 5))
    plt.plot(loss_history_total, label='Total Loss')
    plt.plot(loss_history_phys, label='PDE Loss')
    plt.plot(loss_history_bc, label='BC Loss')
    plt.yscale('log')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Loss trends over training")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    total_epochs = len(loss_history_total)
    avg_time = sum(times_per_epoch) / total_epochs
    print(f"\n✅ Training complete: {total_epochs} epochs")
    print(f"⏱ Average time per epoch: {avg_time:.4f} seconds")

    return None

#### **Validation**

In [ ]:
def validation(model, epochs=10, lambda_bc=1.0):
   
    mse_loss = nn.MSELoss()
    loss_bc = 1
    loss_phys = 1 

    for epoch in range(epochs):   
             
        x_in, mu_in, xb, mu_b = generate_data()
        
        model.eval()

        # Phiycial Loss (residual PDE)
        loss_phys = mse_loss(R(x_in, mu_in, model), torch.zeros_like(x_in[:, :1]))

        # Boundary Loss (Dirichlet)
        u_b = model(xb, mu_b)
        loss_bc = mse_loss(u_b, torch.zeros_like(u_b))

        # Total Loss 
        loss_total = loss_phys + lambda_bc * loss_bc

        print(f"Epoch {epoch} - Loss: {loss_total.item():.6e}  [PDE: {loss_phys.item():.2e}, BC: {loss_bc.item():.2e}]")

In [ ]:
# 1- PINN40
class PINN40(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(4, 40), nn.Tanh(),
        nn.Linear(40, 40), nn.Tanh(),
        nn.Linear(40, 40), nn.Tanh(),
        nn.Linear(40, 40), nn.Tanh(),
        nn.Linear(40, 1)
    )
  def forward(self, x, mu):
    return self.net(torch.cat((x, mu), dim=1))

PINN40 = PINN40()
PINN40.load_state_dict(torch.load("./PINN40.pth", map_location="cpu"))
validation(PINN40)

In [ ]:
# 2- PINN60 
class PINN60(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(4, 60), nn.Tanh(),
        nn.Linear(60, 60), nn.Tanh(),
        nn.Linear(60, 60), nn.Tanh(),
        nn.Linear(60, 60), nn.Tanh(),
        nn.Linear(60, 1)
    )
  def forward(self, x, mu):
    return self.net(torch.cat((x, mu), dim=1))

PINN60 = PINN60()
PINN60.load_state_dict(torch.load("./PINN60.pth", map_location="cpu"))
validation(PINN60)

In [ ]:
# 3- PINN80
class PINN80(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(4, 80), nn.Tanh(),
        nn.Linear(80, 80), nn.Tanh(),
        nn.Linear(80, 80), nn.Tanh(),
        nn.Linear(80, 80), nn.Tanh(),
        nn.Linear(80, 1)
    )
  def forward(self, x, mu):
    return self.net(torch.cat((x, mu), dim=1))

PINN80 = PINN80()
PINN80.load_state_dict(torch.load("./PINN80.pth", map_location="cpu"))
validation(PINN80)

In [ ]:
# 4- PINN128
class PINN128(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(4, 128), nn.Tanh(),
        nn.Linear(128, 128), nn.Tanh(),
        nn.Linear(128, 128), nn.Tanh(),
        nn.Linear(128, 128), nn.Tanh(),
        nn.Linear(128, 1)
    )
  def forward(self, x, mu):
    return self.net(torch.cat((x, mu), dim=1))

PINN128 = PINN128()
PINN128.load_state_dict(torch.load("./PINN128.pth", map_location="cpu"))
validation(PINN128)

In [ ]:
# 5- PINN_ReLU_60
class PINN_ReLU_60(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 60), nn.ReLU(),
            nn.Linear(60, 60), nn.ReLU(),
            nn.Linear(60, 60), nn.ReLU(),
            nn.Linear(60, 60), nn.ReLU(),
            nn.Linear(60, 1)
        )

    def forward(self, x, mu):
        return self.net(torch.cat((x, mu), dim=1))

PINN_ReLU_60 = PINN_ReLU_60()
PINN_ReLU_60.load_state_dict(torch.load("./PINN_ReLU_60.pth", map_location="cpu"))
validation(PINN_ReLU_60)

In [ ]:
# 6- PINNResNet

class ResBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(dim, dim), nn.Tanh(),
            nn.Linear(dim, dim)
        )
    def forward(self, x):
        return x + self.block(x)

class PINNResNet(nn.Module):
    def __init__(self, hidden_dim=60, num_blocks=4):
        super().__init__()
        self.input = nn.Linear(4, hidden_dim)
        self.blocks = nn.Sequential(*[ResBlock(hidden_dim) for _ in range(num_blocks)])
        self.out = nn.Linear(hidden_dim, 1)

    def forward(self, x, mu):
        z = torch.cat((x, mu), dim=1)
        z = torch.tanh(self.input(z))
        z = self.blocks(z)
        return self.out(z)

PINNResNet = PINNResNet()
PINNResNet.load_state_dict(torch.load("./PINNResNet.pth", map_location="cpu"))
validation(PINNResNet)

In [ ]:
# 7- PINN_Tanh_Deep
class PINN_Tanh_Deep(nn.Module):
    def __init__(self):
        super().__init__()
        layers = [nn.Linear(4, 40), nn.Tanh()]
        for _ in range(7):
            layers += [nn.Linear(40, 40), nn.Tanh()]
        layers.append(nn.Linear(40, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x, mu):
        return self.net(torch.cat((x, mu), dim=1))

PINN_Tanh_Deep = PINN_Tanh_Deep()
PINN_Tanh_Deep.load_state_dict(torch.load("./PINN_Tanh_Deep.pth", map_location="cpu"))
validation(PINN_Tanh_Deep)

In [ ]:
# 8- PINN_SiLU_60 
class PINN_SiLU_60(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 60), nn.SiLU(),
            nn.Linear(60, 60), nn.SiLU(),
            nn.Linear(60, 60), nn.SiLU(),
            nn.Linear(60, 60), nn.SiLU(),
            nn.Linear(60, 1)
        )

    def forward(self, x, mu):
        return self.net(torch.cat((x, mu), dim=1))

PINN_SiLU_60 = PINN_SiLU_60()
PINN_SiLU_60.load_state_dict(torch.load("./PINN_SiLU_60.pth", map_location="cpu"))
validation(PINN_SiLU_60)

In [ ]:
# 9- PINN_Sine_60 

class Sine(nn.Module):
    def forward(self, input):
        return torch.sin(input)

class PINN_Sine_60(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 60), Sine(),
            nn.Linear(60, 60), Sine(),
            nn.Linear(60, 60), Sine(),
            nn.Linear(60, 60), Sine(),
            nn.Linear(60, 1)
        )

    def forward(self, x, mu):
        return self.net(torch.cat((x, mu), dim=1))

PINN_Sine_60 = PINN_Sine_60()
PINN_Sine_60.load_state_dict(torch.load("./PINN_Sine_60.pth", map_location="cpu"))
validation(PINN_Sine_60)

In [ ]:
# 10- PINN_ELU_60 TODO
class PINN_ELU_60(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 60), nn.ELU(),
            nn.Linear(60, 60), nn.ELU(),
            nn.Linear(60, 60), nn.ELU(),
            nn.Linear(60, 60), nn.ELU(),
            nn.Linear(60, 1)
        )

    def forward(self, x, mu):
        return self.net(torch.cat((x, mu), dim=1))

PINN_ELU_60 = PINN_ELU_60()
PINN_ELU_60.load_state_dict(torch.load("./PINN_ELU_60.pth", map_location="cpu"))
validation(PINN_ELU_60)

In [ ]:
# 11- PINN_LN_60
class PINN_LN_60(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 60), nn.LayerNorm(60), nn.Tanh(),
            nn.Linear(60, 60), nn.LayerNorm(60), nn.Tanh(),
            nn.Linear(60, 60), nn.LayerNorm(60), nn.Tanh(),
            nn.Linear(60, 60), nn.LayerNorm(60), nn.Tanh(),
            nn.Linear(60, 1)
        )

    def forward(self, x, mu):
        return self.net(torch.cat((x, mu), dim=1))

PINN_LN_60 = PINN_LN_60()
PINN_LN_60.load_state_dict(torch.load("./PINN_LN_60.pth", map_location="cpu"))
validation(PINN_LN_60)

In [ ]:
# 12- PINN_MixAct
class PINN_MixAct(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 60), nn.Tanh(),
            nn.Linear(60, 60), nn.ReLU(),
            nn.Linear(60, 60), nn.Tanh(),
            nn.Linear(60, 60), nn.ReLU(),
            nn.Linear(60, 1)
        )

    def forward(self, x, mu):
        return self.net(torch.cat((x, mu), dim=1))

PINN_MixAct = PINN_MixAct()
PINN_MixAct.load_state_dict(torch.load("./PINN_MixAct.pth", map_location="cpu"))
validation(PINN_MixAct)

#### **Compute the error** on the quadrature points

In [ ]:
# Compute quadrature points
quadrature_points = []
file_path = "./Export/SolutionOnPoints_0.inp" # Take the quadrature poing just computed in the HIGH FIDELITY part (before POD)

with open(file_path, "r") as f:
    next(f)  # No point in the header (first line) of the file
    for line in f:
        parts = line.strip().split()
        if len(parts) < 4:
            continue
        try:
            idx = int(parts[0])
            x = float(parts[1])
            y = float(parts[2])
        except ValueError:
            # Righe non dati, le salto
            continue

        quadrature_points.append((x, y))
quadrature_points = np.array(quadrature_points)  # shape (N,2)
print(f"quadrature_points.shape: {quadrature_points.shape}")

In [ ]:
sample_mu_0 = sample_mu_0.reshape(-1, 1)  # shape: (300, 1)
sample_mu_1 = sample_mu_1.reshape(-1, 1)  # shape: (300, 1)

# Now concatenate column-wise
test_set = np.hstack((sample_mu_0, sample_mu_1))

print(f"test_set.shape: {test_set.shape}")

In [ ]:
def save_csv_file(name_model):
    coords_tensor = torch.tensor(quadrature_points, dtype=torch.float32)  # (N, 2) # (1470, 2)
    
    all_preds = []
    all_u_x = []
    all_u_y = []
    times_pinn = []
    
    for mu in test_set: # (300,2)
        mu0_val, mu1_val = mu
    
        # Separate mu and x to give to the net 
        x_tensor = coords_tensor  # (N, 2)
        mu_tensor = torch.full((coords_tensor.shape[0], 2), fill_value=0.0, dtype=torch.float32)
        mu_tensor[:, 0] = mu0_val
        mu_tensor[:, 1] = mu1_val
    
        x_tensor.requires_grad_(True)
    
        start_time = time.time()
        u = net(x_tensor, mu_tensor)  # (N, 1)
        time_pinn = time.time() - start_time
        times_pinn.append(time_pinn)
    
        u_pred = u.detach().cpu().numpy().flatten()
        all_preds.append(u_pred)
    
        grads = torch.autograd.grad(
            outputs=u.sum(), inputs=x_tensor, create_graph=False, retain_graph=False
        )[0]  # (N, 2)
        u_x = grads[:, 0].detach().cpu().numpy()
        u_y = grads[:, 1].detach().cpu().numpy()
    
        all_u_x.append(u_x)
        all_u_y.append(u_y)
    
    all_preds = np.array(all_preds).T      # (N_punti, N_mu )
    all_u_x = np.array(all_u_x).T          # (N_punti, N_mu )
    all_u_y = np.array(all_u_y).T          # (N_punti, N_mu )
    
    col_names = [f"mu0_{mu[0]:.3f}_mu1_{mu[1]:.3f}" for mu in test_set]
    
    # Save u
    df_u = pd.DataFrame(all_preds, columns=col_names)
    df_u.to_csv(f"pred_u_{name_model}.csv", index=False)
    
    # Save u_x
    df_ux = pd.DataFrame(all_u_x, columns=col_names)
    df_ux.to_csv(f"pred_u_x_{name_model}.csv", index=False) 
    
    # Save u_y
    df_uy = pd.DataFrame(all_u_y, columns=col_names)
    df_uy.to_csv(f"pred_u_y_{name_model}.csv", index=False)
    
    # Save tempi
    df_times = pd.DataFrame(times_pinn)
    df_times.to_csv(f"time_pinn_{name_model}.csv", index=False)
    
    print("Save: u, u_x, u_y, time")

    return None

In [ ]:
def csv(name_model):
    # Carica il CSV delle predizioni pred_u, pred_u_x, pred_u_y, time_pinn
    df = pd.read_csv(f"pred_u_{name_model}.csv")
    
    # Estrai i valori come array NumPy: shape (N_punti, N_mu)
    sol_quad_pinn = df.values.T
    print(np.array(sol_quad_pinn).shape)
    
    print(f"Caricate predizioni: shape = {sol_quad_pinn.shape}")
    
    # Carica il CSV della derivata rispetto a x
    df = pd.read_csv(f"pred_u_x_{name_model}.csv")
    
    # Estrai i valori come array NumPy: shape (N_punti, N_mu)
    der_x_quad_pinn = df.values.T
    print(np.array(der_x_quad_pinn).shape)
    
    # Carica il CSV della derivata rispetto a y
    df = pd.read_csv(f"pred_u_y_{name_model}.csv")
    
    # Estrai i valori come array NumPy: shape (N_punti, N_mu)
    der_y_quad_pinn = df.values.T
    print(np.array(der_y_quad_pinn).shape)
    
    # Carica il CSV dei tempi
    df = pd.read_csv(f"time_pinn_{name_model}.csv")
    return sol_quad_pinn

In [ ]:
def compute_L2_H1_errors_separated(HF_u_set, HF_ux_set, HF_uy_set, POD_u_set, POD_ux_set, POD_uy_set, weights=None):
    l2_errors = []
    l2_rel_errors = []

    for u_HF, u_POD in zip(HF_u_set,POD_u_set):

        # Compute errors
        err_u = u_POD - u_HF

        if weights is not None:
            l2 = np.sqrt(np.sum(weights * err_u**2))
            
            l2_rel = l2/np.sqrt(np.sum(weights * u_HF**2))
            
        else:
            l2 = np.linalg.norm(err_u)
           

        l2_errors.append(l2)
       
        l2_rel_errors.append(l2_rel)
        
    avg_l2 = np.mean(l2_errors)
   
    avg_l2_rel = np.mean(l2_rel_errors)
    
    print(f"Error L^2 (abs) computed on quadrature points: {avg_l2:.4e}")
    print(f"Error L^2 (rel) computed on quadrature points: {avg_l2_rel:.4e}")
    
    return {
        "Error L^2 (abs) computed on quadrature points": avg_l2,
        "Error L^2 (rel) computed on quadrature points": avg_l2_rel,
    }

In [ ]:
# 1- PINN40
class PINN40(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(4, 40), nn.Tanh(),
        nn.Linear(40, 40), nn.Tanh(),
        nn.Linear(40, 40), nn.Tanh(),
        nn.Linear(40, 40), nn.Tanh(),
        nn.Linear(40, 1)
    )
  def forward(self, x, mu):
    return self.net(torch.cat((x, mu), dim=1))


# Caricamento del modello
net = PINN40()
net.load_state_dict(torch.load("PINN40.pth", map_location="cpu")) # Load the net
net.eval() # Put the net in eval mode

name_model = "PINN40"

save_csv_file(name_model)

sol_quad_pinn = csv(name_model)

der_x_quad_hf = 0
der_y_quad_hf = 0
der_x_quad_pinn = 0
der_y_quad_pinn = 0

errors_u_set = compute_L2_H1_errors_separated(sol_quad_hf,der_x_quad_hf,der_y_quad_hf, sol_quad_pinn,der_x_quad_pinn,der_y_quad_pinn, weights=quadratureWeights)

In [ ]:
# 2- PINN60
class PINN60(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(4, 60), nn.Tanh(),
        nn.Linear(60, 60), nn.Tanh(),
        nn.Linear(60, 60), nn.Tanh(),
        nn.Linear(60, 60), nn.Tanh(),
        nn.Linear(60, 1)
    )
  def forward(self, x, mu):
    return self.net(torch.cat((x, mu), dim=1))

net = PINN60()
net.load_state_dict(torch.load("PINN60.pth", map_location="cpu")) # CHANGE HERE THE NAME

net.eval() # Put the net in eval mode

name_model = "PINN60"
save_csv_file(name_model)

sol_quad_pinn = csv(name_model)

der_x_quad_hf = 0
der_y_quad_hf = 0
errors_u_set = compute_L2_H1_errors_separated(sol_quad_hf,der_x_quad_hf,der_y_quad_hf, sol_quad_pinn,der_x_quad_pinn,der_y_quad_pinn, weights=quadratureWeights)

In [ ]:
# 3- PINN80
class PINN80(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(4, 80), nn.Tanh(),
        nn.Linear(80, 80), nn.Tanh(),
        nn.Linear(80, 80), nn.Tanh(),
        nn.Linear(80, 80), nn.Tanh(),
        nn.Linear(80, 1)
    )
  def forward(self, x, mu):
    return self.net(torch.cat((x, mu), dim=1))

# Caricamento del modello
net = PINN80()
net.load_state_dict(torch.load("PINN80.pth", map_location="cpu")) # Load the net

net.eval() # Put the net in eval mode

name_model = "PINN80"

save_csv_file(name_model)

sol_quad_pinn = csv(name_model)

der_x_quad_hf = 0
der_y_quad_hf = 0
errors_u_set = compute_L2_H1_errors_separated(sol_quad_hf,der_x_quad_hf,der_y_quad_hf, sol_quad_pinn,der_x_quad_pinn,der_y_quad_pinn, weights=quadratureWeights)

In [ ]:
# 4- PINN128
class PINN128(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(4, 128), nn.Tanh(),
        nn.Linear(128, 128), nn.Tanh(),
        nn.Linear(128, 128), nn.Tanh(),
        nn.Linear(128, 128), nn.Tanh(),
        nn.Linear(128, 1)
    )
  def forward(self, x, mu):
    return self.net(torch.cat((x, mu), dim=1))

net = PINN128()
net.load_state_dict(torch.load("PINN128.pth", map_location="cpu")) # CHANGE HERE THE NAME

net.eval() # Put the net in eval mode

model_name = "PINN128"
save_csv_file(model_name)

sol_quad_pinn = csv(model_name)

der_x_quad_hf = 0
der_y_quad_hf = 0
errors_u_set = compute_L2_H1_errors_separated(sol_quad_hf,der_x_quad_hf,der_y_quad_hf, sol_quad_pinn,der_x_quad_pinn,der_y_quad_pinn, weights=quadratureWeights)

In [ ]:
# 5- PINN_ReLU_60 
class PINN_ReLU_60(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 60), nn.ReLU(),
            nn.Linear(60, 60), nn.ReLU(),
            nn.Linear(60, 60), nn.ReLU(),
            nn.Linear(60, 60), nn.ReLU(),
            nn.Linear(60, 1)
        )

    def forward(self, x, mu):
        return self.net(torch.cat((x, mu), dim=1))

net = PINN_ReLU_60()
net.load_state_dict(torch.load("PINN_ReLU_60.pth", map_location="cpu")) # CHANGE HERE THE NAME

net.eval() # Put the net in eval mode

name_model = "PINN_ReLU_60.pth"
save_csv_file(name_model)
sol_quad_pinn = csv(name_model)

der_x_quad_hf = 0
der_y_quad_hf = 0
errors_u_set = compute_L2_H1_errors_separated(sol_quad_hf,der_x_quad_hf,der_y_quad_hf, sol_quad_pinn,der_x_quad_pinn,der_y_quad_pinn, weights=quadratureWeights)

In [ ]:
# 6- PINNResNet
class PINNResNet(nn.Module):
    def __init__(self, hidden_dim=60, num_blocks=4):
        super().__init__()
        self.input = nn.Linear(4, hidden_dim)
        self.blocks = nn.Sequential(*[ResBlock(hidden_dim) for _ in range(num_blocks)])
        self.out = nn.Linear(hidden_dim, 1)

    def forward(self, x, mu):
        z = torch.cat((x, mu), dim=1)
        z = torch.tanh(self.input(z))
        z = self.blocks(z)
        return self.out(z)

net = PINNResNet()
net.load_state_dict(torch.load("PINNResNet.pth", map_location="cpu")) # CHANGE HERE THE NAME

net.eval() # Put the net in eval mode

name_model = "PINNResNet.pth"

save_csv_file(name_model)

sol_quad_pinn = csv(name_model)

der_x_quad_hf = 0
der_y_quad_hf = 0
errors_u_set = compute_L2_H1_errors_separated(sol_quad_hf,der_x_quad_hf,der_y_quad_hf, sol_quad_pinn,der_x_quad_pinn,der_y_quad_pinn, weights=quadratureWeights)

In [ ]:
# 7- PINN_Tanh_Deep
class PINN_Tanh_Deep(nn.Module):
    def __init__(self):
        super().__init__()
        layers = [nn.Linear(4, 40), nn.Tanh()]
        for _ in range(7):
            layers += [nn.Linear(40, 40), nn.Tanh()]
        layers.append(nn.Linear(40, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x, mu):
        return self.net(torch.cat((x, mu), dim=1))

net = PINN_Tanh_Deep()
net.load_state_dict(torch.load("PINN_Tanh_Deep.pth", map_location="cpu")) # CHANGE HERE THE NAME

net.eval() # Put the net in eval mode

name_model = "PINN_Tanh_Deep"

save_csv_file(name_model)

sol_quad_pinn = csv(name_model)

der_x_quad_hf = 0
der_y_quad_hf = 0
errors_u_set = compute_L2_H1_errors_separated(sol_quad_hf,der_x_quad_hf,der_y_quad_hf, sol_quad_pinn,der_x_quad_pinn,der_y_quad_pinn, weights=quadratureWeights)

In [ ]:
# 8- PINN_SiLU_60
class PINN_SiLU_60(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 60), nn.SiLU(),
            nn.Linear(60, 60), nn.SiLU(),
            nn.Linear(60, 60), nn.SiLU(),
            nn.Linear(60, 60), nn.SiLU(),
            nn.Linear(60, 1)
        )

    def forward(self, x, mu):
        return self.net(torch.cat((x, mu), dim=1))

net = PINN_SiLU_60()
net.load_state_dict(torch.load("PINN_SiLU_60.pth", map_location="cpu")) # CHANGE HERE THE NAME

name_model = "PINN_SiLU_60.pth"
net.eval() # Put the net in eval mode

save_csv_file(name_model)

sol_quad_pinn = csv(name_model)

der_x_quad_hf = 0
der_y_quad_hf = 0
errors_u_set = compute_L2_H1_errors_separated(sol_quad_hf,der_x_quad_hf,der_y_quad_hf, sol_quad_pinn,der_x_quad_pinn,der_y_quad_pinn, weights=quadratureWeights)

In [ ]:
# 9- PINN_Sine_60 TODO
class PINN_Sine_60(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 60), Sine(),
            nn.Linear(60, 60), Sine(),
            nn.Linear(60, 60), Sine(),
            nn.Linear(60, 60), Sine(),
            nn.Linear(60, 1)
        )

    def forward(self, x, mu):
        return self.net(torch.cat((x, mu), dim=1))

net = PINN_Sine_60()
net.load_state_dict(torch.load("PINN_Sine_60.pth", map_location="cpu")) # CHANGE HERE THE NAME

net.eval() # Put the net in eval mode

name_model = "PINN_Sine_60"
save_csv_file(name_model)

sol_quad_pinn = csv(name_model)

der_x_quad_hf = 0
der_y_quad_hf = 0
errors_u_set = compute_L2_H1_errors_separated(sol_quad_hf,der_x_quad_hf,der_y_quad_hf, sol_quad_pinn,der_x_quad_pinn,der_y_quad_pinn, weights=quadratureWeights)

In [ ]:
# 10- PINN_ELU_60
class PINN_ELU_60(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 60), nn.ELU(),
            nn.Linear(60, 60), nn.ELU(),
            nn.Linear(60, 60), nn.ELU(),
            nn.Linear(60, 60), nn.ELU(),
            nn.Linear(60, 1)
        )

    def forward(self, x, mu):
        return self.net(torch.cat((x, mu), dim=1))

net = PINN_ELU_60()
net.load_state_dict(torch.load("PINN_ELU_60.pth", map_location="cpu")) # CHANGE HERE THE NAME

net.eval() # Put the net in eval mode

name_model = "PINN_ELU_60"
save_csv_file(name_model)

sol_quad_pinn = csv(name_model)

der_x_quad_hf = 0
der_y_quad_hf = 0
errors_u_set = compute_L2_H1_errors_separated(sol_quad_hf,der_x_quad_hf,der_y_quad_hf, sol_quad_pinn,der_x_quad_pinn,der_y_quad_pinn, weights=quadratureWeights)

In [ ]:
# 11- PINN_LN_60
class PINN_LN_60(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 60), nn.LayerNorm(60), nn.Tanh(),
            nn.Linear(60, 60), nn.LayerNorm(60), nn.Tanh(),
            nn.Linear(60, 60), nn.LayerNorm(60), nn.Tanh(),
            nn.Linear(60, 60), nn.LayerNorm(60), nn.Tanh(),
            nn.Linear(60, 1)
        )

    def forward(self, x, mu):
        return self.net(torch.cat((x, mu), dim=1))

net = PINN_LN_60()
net.load_state_dict(torch.load("PINN_LN_60.pth", map_location="cpu")) # CHANGE HERE THE NAME

net.eval() # Put the net in eval mode

model_name = "PINN_LN_60"
save_csv_file(model_name)
sol_quad_pinn = csv(model_name)

der_x_quad_hf = 0
der_y_quad_hf = 0
errors_u_set = compute_L2_H1_errors_separated(sol_quad_hf,der_x_quad_hf,der_y_quad_hf, sol_quad_pinn,der_x_quad_pinn,der_y_quad_pinn, weights=quadratureWeights)

In [ ]:
# 12- PINN_MixAct
class PINN_MixAct(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 60), nn.Tanh(),
            nn.Linear(60, 60), nn.ReLU(),
            nn.Linear(60, 60), nn.Tanh(),
            nn.Linear(60, 60), nn.ReLU(),
            nn.Linear(60, 1)
        )

    def forward(self, x, mu):
        return self.net(torch.cat((x, mu), dim=1))

net = PINN_MixAct()
net.load_state_dict(torch.load("PINN_MixAct.pth", map_location="cpu")) # CHANGE HERE THE NAME

net.eval() # Put the net in eval mode

model_name = "PINN_MixAct"
save_csv_file(model_name)

sol_quad_pinn = csv(model_name)

der_x_quad_hf = 0
der_y_quad_hf = 0
errors_u_set = compute_L2_H1_errors_separated(sol_quad_hf,der_x_quad_hf,der_y_quad_hf, sol_quad_pinn,der_x_quad_pinn,der_y_quad_pinn, weights=quadratureWeights)